## getting SPoCA annotations from HEK (Heliophysics Event Knowledgebase)  
(more info about HEK: https://link.springer.com/content/pdf/10.1007/978-1-4614-3673-7_5.pdf)  
   
In this notebook, we present functions with script that helped us get SPoCA annotations for SOHO images  
Main function is **make_spoca_imgs()** that uses other defined functions.  
**Main script that uses make_spoca_imgs()** function is at the bottom   
We also provide additional script that helped us move images to folder


In [11]:
import glob
from sunpy.net import attrs as a
from sunpy.net import Fido
# sunpy lib may not be resolved by VS code but works fine
import datetime
from PIL import Image, ImageDraw
from PIL import Image
from tqdm.notebook import tqdm, trange

In [7]:
def get_date_hours(path):
    """gets date and horus string for helioviewerClient from filepath

    Args:
        path (string): filepath from which we need date 

    Returns:
        string: date in format %Y/%m/%d %H:%M:%S
    """

    date_str = path.split("/")[-1].split("_")[0]
    time_str = path.split("/")[-1].split("_")[1]
    dt = datetime.datetime.strptime(date_str + time_str, "%Y%m%d%H%M")
    return dt.strftime("%Y/%m/%d %H:%M:%S")

In [4]:
def get_coords(polygon_result):
    """gets coords for annotation from Fido search result

    Args:
        polygon_result (list): list of coordinates of ONE event from Fido search result
    """
    CENTER = 1328

    length = len(polygon_result)
    polygon_coord = polygon_result[9:length - 2]

    coords = polygon_coord.split(",")
    single_coord = []

    for coord in coords:
        single_coord.append(coord.split(" "))

    one_coords = []
    for coord in single_coord:
        one_coords.append(list(map(float, coord)))

    for i in range(len(one_coords)):
        one_coords[i][0] = CENTER + one_coords[i][0]
        one_coords[i][1] = CENTER - one_coords[i][1]

    for i in range(len(one_coords)):
        one_coords[i] = tuple(one_coords[i])

    one_coords = tuple(one_coords)
    return(one_coords)

In [5]:
def get_anotations(path, event_type):
    """gets spoca annotation for file. Uses Fido to shearch for SPoCA annotations, then get_coords() to extract coordiantions to list,
    so we can use PIL and its ImageDraw to draw polygon of SPoCA annotations coordinates on image

    Args:
        path (string): path to image file

        event_type (string): "CH" or "AR"

    Returns:
        list: coordinates of SPoCA annotation for PIL to draw polygon with them
    """
    time = get_date_hours(path)

    tstart = time
    tend = time
    result = Fido.search(a.Time(tstart,tend) , a.hek.EventType(event_type))
    anotations = []

    # with result["hek"]["hpc_boundcc"] we can acces what Fido has found in HEK database
    # the result is in form of list of tuples of coordinates where is SPoCA annotation on the image
    # as there are more events that day on sun disk, len(result["hek"]) is bigger. if len(result["hek"]) == 0, there are no event with SPoCA annotations
    # print(result["hek"]["hpc_boundcc"])
    if len(result["hek"]) == 0:
        return anotations

    # extract annotations from Fido search
    # loop every event on sun disk and get coords
    for i in range(len(result["hek"]["bound_chaincode"])):
        polygon_result = result["hek"]["bound_chaincode"][i]
        if polygon_result == "":
            continue
        anotations.append(get_coords(polygon_result))

    return anotations

In [35]:
def make_spoca_imgs(path, save_path, event):
    """Main function that uses ither defined functions. Saves imgs with spoca annotations

    Args:
        path (string): path to image for which it should find and draw SPoCA annotations.

        save_path (string): path where to save file of SPoCA annotations. It will save black and white images where white regions are
        annotation os selected event

        event (string): either "CH" for coronal holes or "AR" for active regions
    """
    IMG_SIZE = 2656   #2656
    #center = 1280 + 48     #1328

    # finds annotation in HEK
    annotations = get_anotations(path, event)

    if len(annotations) == 0:
        return

    # creates black image
    img = Image.new("RGBA", [IMG_SIZE, IMG_SIZE], color = "BLACK")
    # next uncommented code would create copy of original image of suns disk and draw SPoCA annotations on copy of original image.
    # img = Image.open(path).resize((IMG_SIZE, IMG_SIZE))
    draw = ImageDraw.Draw(img)

    # draws white regions that represents SPoCA annotation
    for annotation in annotations:
        draw.polygon(annotation, outline="white", width=5, fill="white")

    resized = img.resize((1024,1024))

    image_name_clean = path[-29: -4]
    download_path = save_path + image_name_clean + ".png"

    resized.save(download_path)

In [8]:
def compare_masks_imgs(path, save_path, all_masks):
    image_name_clean = path[-29:]

    if image_name_clean in all_masks:
        img = Image.open(path)
        download_path = save_path + image_name_clean
        img.save(download_path)

<img src="../figures/spoca-segmentacia.png"  width="600" height="600">  

slnecny disk s najdenou SPoCA annotaciou na JENEJ koronalnej diere

## Get SPoCA annotations of CH or AR for every year  
Main script to get annotations.  
SPoCA for coronal holes = 2011 - today  
SPoCA for Active regions = 1996 - today

In [56]:
year_to_find = 1996
# change file path
save_path = "/Users/majirky/Desktop/ar_spoca/ARspocaBWimgs/"


for i in range(26):

    # change path for imgs on which you want to find SPoCA annotations
    # if you do not have folders by years, but one big folder with imgs from every year, you should delete for loop
    resource_path = f"/Users/majirky/Desktop/slnko_ar/arfotky_{year_to_find}/*"
    print(f"rok hladania: {year_to_find}")


    paths = []
    for name in glob.glob(resource_path):
        paths.append(name)

    for path in tqdm(paths):
        make_spoca_imgs(path, save_path, "AR")

    year_to_find = year_to_find + 1



rok hladania: 2019


  0%|          | 0/282 [00:00<?, ?it/s]

rok hladania: 2020


  0%|          | 0/326 [00:00<?, ?it/s]

rok hladania: 2021


  0%|          | 0/303 [00:00<?, ?it/s]

# ADDITIONALS

## MOVE IMGS FROM YEARS TO SINGLE DIRC
we had folders in which where images of sun by year. We used this short script to move those images from folders to one big folder

In [12]:
save_path = "/Users/majirky/Desktop/slnko_ar/arfotky_96-21/"


for i in tqdm(range(1996, 2022)):

    resource_path = f"/Users/majirky/Desktop/slnko_ar/arfotky_{i}/*"
    print(f"rok ukladania: {i}")

    paths = []
    for name in glob.glob(resource_path):
        image_name_clean = name[-29: -4]
        download_path = save_path + image_name_clean + ".png"

        img = Image.open(name)
        img.save(download_path)


  0%|          | 0/26 [00:00<?, ?it/s]

rok ukladania: 1996
rok ukladania: 1997
rok ukladania: 1998
rok ukladania: 1999
rok ukladania: 2000
rok ukladania: 2001
rok ukladania: 2002
rok ukladania: 2003
rok ukladania: 2004
rok ukladania: 2005
rok ukladania: 2006
rok ukladania: 2007
rok ukladania: 2008
rok ukladania: 2009
rok ukladania: 2010
rok ukladania: 2011
rok ukladania: 2012
rok ukladania: 2013
rok ukladania: 2014
rok ukladania: 2015
rok ukladania: 2016
rok ukladania: 2017
rok ukladania: 2018
rok ukladania: 2019
rok ukladania: 2020
rok ukladania: 2021
